## 加载数据

In [1]:
import pandas as pd
from sklearn.base import clone
from config import Datasets, Datasets_test
from feature_selection import FeatureSelection, non_dominated_sort, train_and_test
from skfeature.function.similarity_based import fisher_score
from skfeature.function.information_theoretical_based import CIFE
from skfeature.function.statistical_based import chi_square
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
import warnings

warnings.filterwarnings("ignore")  # 忽略警告

# 数据集
fs = FeatureSelection(Datasets_test)
# 创建带有列名的空DataFrame
df = pd.DataFrame(columns=['数据集', '类分布', '特征数量', '特征选择数量', '原始', 'SMOTE', '特征选择+SMOTE'])
for i in range(len(Datasets_test)):
    # 数据预处理
    fs.pre_process(Datasets_test[i], random_state=42)
    print(f"{i + 1}th dataset: {fs.dataset.DATASETNAME}")
    fs.display_distribution()

    # 特征选择（三种不同的选择算法）
    idx_1 = fs.feature_selection(fisher_score.fisher_score, mode='index')
    # print(idx_1)
    idx_2 = fs.feature_selection(chi_square.chi_square, mode='index')
    # print(idx_2)
    idx_3 = fs.feature_selection(CIFE.cife, mode='index', n_selected_features=fs.x_train.shape[1])
    # print(idx_3)

    # 非支配排序（三种算法的特征排名）
    all_fronts = non_dominated_sort(idx_1, idx_2, idx_3)

    # 前后结果对比（原始数据、SMOTE、特征选择+SMOTE）
    model = MLPClassifier(hidden_layer_sizes=(fs.dataset.HIDDEN_SIZE,), max_iter=fs.dataset.MAX_ITER,
                          random_state=42, learning_rate_init=fs.dataset.LEARNING_RATE)
    res_1 = train_and_test(clone(model), fs.x_train, fs.x_test, fs.y_train, fs.y_test)
    print(f"原始：{res_1}")
    x_train, y_train = SMOTE(random_state=42, k_neighbors=fs.dataset.K_NEIGHBORS).fit_resample(fs.x_train, fs.y_train)
    res_2 = train_and_test(clone(model), x_train, fs.x_test, y_train, fs.y_test)
    print(f"SMOTE：{res_2}")
    x_train, y_train = SMOTE(random_state=42, k_neighbors=fs.dataset.K_NEIGHBORS).fit_resample(
        fs.x_train[:, all_fronts[0]], fs.y_train)
    res_3 = train_and_test(clone(model), x_train, fs.x_test[:, all_fronts[0]], y_train, fs.y_test)
    print(f"特征选择+SMOTE：{res_3}")
    # 保存结果
    df.loc[i] = [fs.dataset.DATASETNAME, fs.distribution, fs.x_train.shape[1],
                 len(all_fronts[0]), res_1, res_2, res_3]
#  保存结果
df.to_csv('feature_selection_result.csv', encoding='utf_8_sig', index=False)

1th dataset: GLIOMA.mat
trainset distribution: [10  5 10 10]
testset distribution: [4 2 4 5]
number of feature: 4434
原始：(np.float64(0.840896), np.float64(0.9625), 0.875)
SMOTE：(np.float64(0.880112), np.float64(0.979167), 0.881944)
特征选择+SMOTE：(np.float64(0.782542), np.float64(0.891667), 0.781439)
2th dataset: Lung.mat
trainset distribution: [97 12 15 14  4]
testset distribution: [42  5  6  6  2]
number of feature: 3312
原始：(np.float64(0.974968), np.float64(0.996111), 0.885437)
SMOTE：(np.float64(0.995192), np.float64(0.998214), 0.979409)
特征选择+SMOTE：(np.float64(0.929667), np.float64(0.98869), 0.839487)
3th dataset: Ovarian.mat
trainset distribution: [113  64]
testset distribution: [49 27]
number of feature: 15154
原始：(np.float64(0.971242), np.float64(0.999244), 0.971277)
SMOTE：(np.float64(0.96225), np.float64(1.0), 0.970769)
特征选择+SMOTE：(np.float64(0.981307), np.float64(1.0), 0.985516)
4th dataset: Semeion.mat
trainset distribution: [1004  111]
testset distribution: [431  47]
number of featu